**INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on the dataset you collected from assignment two. You may use scipy and numpy package in this assignment.



In [4]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!pip install options
!pip install requests
from selenium import webdriver
import pandas as pd
import csv
import time
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from tabulate import tabulate
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import numpy as np

     |████████████████████████████████| 911kB 6.4MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [5]:
!pip install stopwords
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
pages=["https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"]
for page in pages:
  scrape = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  scrape.get(page)
  for i in range(3):
    button=scrape.find_element_by_xpath('//*[@id="load-more-trigger"]').click()
    time.sleep(3)
    r=[]
    items=scrape.find_elements_by_class_name("content")
    for item in items:
      r.append(item.find_element_by_css_selector('.text.show-more__control').text)
      reviews_df=pd.DataFrame(r, columns = ['Review_by_the_users'])
      reviews_df.to_csv('reviews_of_joker.csv')
print(reviews_df) #extracting 100 reviews 
#reviews_df
string = open('reviews_of_joker.csv', newline='').read()
no_specials_string = re.sub('[!#?,.:";-]', '', string)
reviews_df['Review_by_the_users'] = reviews_df['Review_by_the_users'].str.replace(r"\W", " ").str.strip() # removing noise
reviews_df['Review_by_the_users']= reviews_df['Review_by_the_users'].str.replace(r'\d+',"")#Removed the numbers
To_lower_case = reviews_df['Review_by_the_users'].str.lower().to_frame() # converted to lowercase
To_lower_case.to_csv('reviews_of_joker1.csv')
print(To_lower_case )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


                                  Review_by_the_users
0   Every once in a while a movie comes, that trul...
1   This is a movie that only those who have felt ...
2   Truly a masterpiece, The Best Hollywood film o...
3   Joaquin Phoenix gives a tour de force performa...
4   Most of the time movies are anticipated like t...
..                                                ...
95  Heath great in dark Knight, even liked Jared L...
96  I was very excited about this movie when it ca...
97  There is doing an homage and then there is bor...
98  Disturbing. Just got out of the movie. My humo...
99  What a legendary character and what a legendar...

[100 rows x 1 columns]
                                  Review_by_the_users
0   every once in a while a movie comes  that trul...
1   this is a movie that only those who have felt ...
2   truly a masterpiece  the best hollywood film o...
3   joaquin phoenix gives a tour de force performa...
4   most of the time movies are anticipated like t...
..  

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [8]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import re
stem1=PorterStemmer()
stem2=[]
stem3=[]
lem1 = WordNetLemmatizer() 
Lem2=[]
lem3=[]
stop=stopwords.words("english")
for i in reviews_df['Review_by_the_users']: # Stop Words, Lemmatization, stemming
    words=word_tokenize(i)
    for j in words:
        if j not in stop:
            stemming=stem1.stem(j.lower())
            stem2.append(stemming)
            lemwords=lem1.lemmatize(j.lower())
            Lem2.append(lemwords)
    stem3.append(' '.join(stem2))
    lem3.append(' '.join(Lem2))
    stem2.clear()
    Lem2.clear()
reviews_df['LEM']=pd.DataFrame(lem3)
reviews_df['STEM']=pd.DataFrame(stem3)
reviews_df.to_csv('reviews_of_joker2.csv')
reviews_df

,Review_by_the_users,LEM,STEM
0,Every once in a while a movie comes that trul...,every movie come truly make impact joaquin per...,everi movi come truli make impact joaquin perf...
1,This is a movie that only those who have felt ...,this movie felt alone isolated truly relate yo...,thi movi felt alon isol truli relat you unders...
2,Truly a masterpiece The Best Hollywood film o...,truly masterpiece the best hollywood film one ...,truli masterpiec the best hollywood film one b...
3,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix give tour de force performance...,joaquin phoenix give tour de forc perform fear...
4,Most of the time movies are anticipated like t...,most time movie anticipated like end falling s...,most time movi anticip like end fall short way...
...,...,...,...
95,Heath great in dark Knight even liked Jared L...,heath great dark knight even liked jared leto ...,heath great dark knight even like jare leto su...
96,I was very excited about this movie when it ca...,i excited movie came theater raving review i e...,i excit movi came theater rave review i even e...
97,There is doing an homage and then there is bor...,there homage borrowing wholesale material this...,there homag borrow wholesal materi thi film li...
98,Disturbing Just got out of the movie My humo...,disturbing just got movie my humor much better...,disturb just got movi my humor much better see...


In [9]:
with open('reviews_of_joker2.csv', 'r') as csv_file:
  data = pd.read_csv(csv_file, low_memory=False)
vector = CountVectorizer(ngram_range=(3,3), analyzer='word')
transform = vector.fit_transform(data['Review_by_the_users'].values.astype('U')) 
calculating_frequency = sum(transform).toarray()[0]
output = pd.DataFrame(calculating_frequency, index=vector.get_feature_names(), columns=['frequency of all the N-grams where (N=3)'])
output

,frequency of all the N-grams where (N=3)
able to move,1
able to ponder,1
able to unlock,1
about all the,1
about brilliant acting,1
...,...
yours bs should,1
yourself and the,1
yourself favor and,2
youth currently being,1


In [10]:
with open('reviews_of_joker2.csv', 'r', encoding='utf-8') as file:
  data1 = file.read()
bi = TextBlob(data1)
bigrams = bi.ngrams(2)
gram={ }
probability={ }
for bigram in bigrams:
  bigram=tuple(bigram)
  gram[bigram]=gram.get(bigram,0)+1
for a, b in gram.items():
  probability[a]=b/bi.words.count(a[0])
output1 = pd.DataFrame.from_dict(probability, orient = 'index', columns=['probabilities for all the bigrams'])
output1['frequency for all the bigrams']=[b for b in gram.values()]
output1

,probabilities for all the bigrams,frequency for all the bigrams
"(Review_by_the_users, LEM)",1.000000,1
"(LEM, STEM)",1.000000,1
"(STEM, 0)",1.000000,1
"(0, Every)",1.000000,1
"(Every, once)",0.071429,1
...,...,...
"(movi, total)",0.004405,1
"(total, worth)",0.200000,1
"(worth, thi)",0.066667,1
"(movi, deserv)",0.004405,1


In [11]:
nlp = spacy.load("en")
file = open('reviews_of_joker2.csv', 'r')
data2 = nlp(file.read())
all_the_noun_phrases = []
for noun in data2.noun_chunks:
  all_the_noun_phrases.append(noun.text)
print(all_the_noun_phrases)
output2 = pd.DataFrame(all_the_noun_phrases, columns = ['all_the_noun_phrases'])
vector2 = CountVectorizer(ngram_range=(3,3), analyzer='word')
transform2 = vector2.fit_transform(output2['all_the_noun_phrases'].values.astype('U')) 
calculating_frequency2 = sum(transform2).toarray()[0]
output3 = pd.DataFrame(calculating_frequency2, index=vector2.get_feature_names(), columns=['all_the_noun_phrases_Frequency'])
output3['all_the_noun_phrases_Probabilities'] = output3 / output3.max()
#output4 = output3.astype(object).transpose() 
output3

['Review_by_the_users', 'LEM', 'STEM', 'a while', 'a movie', 'an impact', 'Joaquin s performance', 'scenography', 'it', 'brilliance', 'Grotesque  haunting', 'cringy', 'times', 'you', 'an eye', 'it', 'seriously funny moments', 'multiple emotions', 'the same time', 'a typical action', 'predictable super hero movie', 'it', 'a proper psychological thriller drama', 'the single best character development', 'I', 'every movie', 'impact joaquin performance scenography brilliance', 'cringy hard watch time', 'blink eye', 'tragic seriously funny moment emotional rollercoaster', 'far typical action', 'predictable super hero movie proper psychological thriller drama', 'single best character development', 'i', 'everi movi', 'truli', 'scenographi brillianc grotesqu haunt cringi', 'hard watch', 'time mesmer blink eye', 'typic action riddl', 'proper psycholog thriller drama', 'singl best charact', 'i', '1,This', 'a movie', 'who', 'it', 'You', 'the motive', 'you', 'the character', 'A lot', 'people', 'thi

,all_the_noun_phrases_Frequency,all_the_noun_phrases_Probabilities
11 this movie,1,0.052632
13 the movie,1,0.052632
27 28 29,1,0.052632
28 29 the,1,0.052632
29 the music,1,0.052632
...,...,...
your superhero movi,1,0.052632
your superhero movie,2,0.105263
your typical super,1,0.052632
youth current indoctrin,1,0.052632


 **Question 2: Undersand TF-IDF and Document representation**
(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program:

(1) To build the documents-terms weights (tf*idf) matrix bold text.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.

In [12]:
import numpy as np
data_frame = pd.read_csv("reviews_of_joker2.csv")
term_frequency2 = data_frame.dropna()
term_frequency1 = (term_frequency2['Review_by_the_users'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index())
term_frequency1.columns = ['All_words','term_frequency']
for i,word in enumerate(term_frequency1['All_words']):
  term_frequency1.loc[i, 'inverse_data_frequency'] = np.log(data_frame.shape[0]/(len(term_frequency2[term_frequency2['Review_by_the_users'].str.contains(word)])))
term_frequency1['term_frequency*inverse_data_frequency'] = term_frequency1['term_frequency'] * term_frequency1['inverse_data_frequency']
term_frequency1.to_csv('term_frequency.csv')
term_frequency1

,All_words,term_frequency,inverse_data_frequency,term_frequency*inverse_data_frequency
0,,1525.0,0.139262,212.374653
1,a,331.0,0.139262,46.095744
2,it,237.0,0.248461,58.885342
3,with,67.0,0.941609,63.087772
4,s,141.0,0.139262,19.635951
...,...,...,...,...
2360,killer,1.0,4.605170,4.605170
2361,Disturbing,1.0,4.605170,4.605170
2362,Just,1.0,3.912023,3.912023
2363,charm,1.0,4.605170,4.605170


In [13]:
with open('term_frequency.csv', 'r') as csv_file:
  data = pd.read_csv(csv_file, low_memory=False)
c1=data['inverse_data_frequency']
print(c1)

0       0.139262
1       0.139262
2       0.248461
3       0.941609
4       0.139262
          ...   
2360    4.605170
2361    4.605170
2362    3.912023
2363    4.605170
2364    4.605170
Name: inverse_data_frequency, Length: 2365, dtype: float64


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(list(term_frequency2.Review_by_the_users.values))
df_tfidf = pd.DataFrame.sparse.from_spmatrix(tfidf, columns=vectorizer.get_feature_names())

from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
cosine_similarities

array([1.        , 0.14410553, 0.13641068, 0.10067316, 0.11013321,
       0.13063479, 0.09606883, 0.1853672 , 0.09766337, 0.11896972,
       0.06202359, 0.11729064, 0.11442291, 0.1338197 , 0.04131649,
       0.09305853, 0.17508238, 0.1379016 , 0.11060042, 0.19847499,
       0.07997237, 0.18383319, 0.11209934, 0.07081299, 0.09513968,
       0.05608577, 0.05647028, 0.04800526, 0.13089251, 0.06935245,
       0.07822066, 0.08089564, 0.128304  , 0.07886419, 0.13518746,
       0.08093556, 0.0927319 , 0.10649756, 0.10855136, 0.09933533,
       0.16318732, 0.06417435, 0.23402736, 0.07322809, 0.13511062,
       0.09567032, 0.13283241, 0.09737302, 0.04263634, 0.16176566,
       0.13323405, 0.16230993, 0.15047871, 0.02473076, 0.09571865,
       0.12449633, 0.19820649, 0.11048785, 0.097147  , 0.19323796,
       0.04958489, 0.03619057, 0.0526207 , 0.11957276, 0.12037296,
       0.07258615, 0.14029902, 0.0364861 , 0.12207867, 0.15228577,
       0.16929715, 0.1076629 , 0.10830112, 0.08509935, 0.07962

In [15]:
related_docs_indices = cosine_similarities.argsort()[::-1]

print('in decreasing order, the most similar Research Paper names based on cosine similarity')
print('-'*80, end='\n\n')
for doc in term_frequency2.Review_by_the_users.values[related_docs_indices[1:]]:
    print(doc, end='\n\n')

in decreasing order, the most similar Research Paper names based on cosine similarity
--------------------------------------------------------------------------------

While the trailers for this movie have may seemed like there was potential for an action packed movie  this movie  clearly before it came to theaters was intended to be a progressive character study  You shouldn t go into this movie expecting intense action or the joker we know of the past  This is a movie about how Arthur Fleck becomes the Joker   I have seen so many bad reviews about how boring or slow the movie is  but I believe it was utter perfection  The first part of the movie we begin to understand Arthur as a person  All of his interactions with everyone in his life  be it good or bad  The whole point of a character study is evolution  And this is far and away one of the greatest evolutions of a character I have ever seen in a movie   Joaquin Phoenix s performance is by far the best acting I have seen in a movie